<a href="https://colab.research.google.com/github/Benyormin/Question_answering/blob/main/AI_TASK_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet rank_bm25


In [ ]:

!pip install --quiet pymupdf
!pip install langchain-google-genai google-generativeai pyngrok langchain_community langchain-google-genai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 43.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.

In [ ]:

import fitz
import re

def _simple_persian_normalize(text: str) -> str:
    # remove BOM and zero-width / invisible chars
    text = text.replace('\ufeff', ' ')
    text = re.sub(r'[\u200c\u200b\u200d\u200e\u200f]+', ' ', text)
    text = re.sub(r'[\x00-\x1f]', ' ', text)   # control chars
    # arabic to persian chars
    text = text.replace('ي', 'ی').replace('ك', 'ک')
    # fix line breaks / hyphenation
    text = re.sub(r'-\s+\n', '', text)
    text = re.sub(r'\n+', ' ', text)
    # unify punctuation spacing
    text = re.sub(r'\s*([،؛:؟.!])\s*', r'\1 ', text)
    # collapse spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Very simple Persian sentence splitter (keeps punctuation as sentence boundary)
_sentence_boundary_re = re.compile(r'(?<=[\.\!\؟\?؛\;\:\n])\s+')

def read_persian_pdf(pdf_path: str):
    """
    Reads selectable Persian PDF and returns:
      pages: list of {'page': int, 'text': str}
      full_text: concatenated normalized text (with page separators)
    This function avoids hazm and other compiled deps.
    """
    doc = fitz.open(pdf_path)
    pages = []
    for i, page in enumerate(doc):
        raw = page.get_text("text") or ""
        norm = _simple_persian_normalize(raw)
        pages.append({'page': i+1, 'text': norm})
    doc.close()
    full_text = "\n\n".join([f"--- page {p['page']} ---\n{p['text']}" for p in pages])
    return pages, full_text



In [ ]:
# Example usage
pdf_file = "/content/Drugs.pdf"   # your file
pages, full_text = read_persian_pdf(pdf_file)
print(f"Extracted pages: {len(pages)}")
print(full_text[:400])


Extracted pages: 50
--- page 1 ---
دیکلوفناک چیست و چه کاربردی دارد؟ دیکلوفناک یکی از پرمصرف ترین داروهای ضدالتهابی غیراستروئیدی است که فعالیت عوامل التهاب زا در بدن را مهار و با ایجاد التهاب و درد مقابله می کند. این دارو به: صورت دو ملح دیکلوفناک سدیم و پتاسیم در بازار موجود است دیکلوفناک سدیم با شروع اثر آهسته و طول اثر بیشتر و دیکلوفناک پتاسیم با شروع اثر سریع و طول اثر کوتاه. این دارو در اشکال دارویی قرص، کپسول، 


In [ ]:
pages[0]

{'page': 1,
 'text': 'دیکلوفناک چیست و چه کاربردی دارد؟ دیکلوفناک یکی از پرمصرف ترین داروهای ضدالتهابی غیراستروئیدی است که فعالیت عوامل التهاب زا در بدن را مهار و با ایجاد التهاب و درد مقابله می کند. این دارو به: صورت دو ملح دیکلوفناک سدیم و پتاسیم در بازار موجود است دیکلوفناک سدیم با شروع اثر آهسته و طول اثر بیشتر و دیکلوفناک پتاسیم با شروع اثر سریع و طول اثر کوتاه. این دارو در اشکال دارویی قرص، کپسول، پماد، شیاف و آمپول برای استفاده خوراکی یا موضعی در تمام داروخانه ها یافت می شود و به محض استفاده، دردهای حاد و ناگهانی نیازمند تسکین سریع درد را آرام می سازد. کاربرد شیاف دیکلوفناک از کاربردهای شیاف مسکن دیکلوفناک می توان به این موارد اشاره کرد: • تسکین سردرد و میگرن: این شیاف جلوی حمالت میگرن را نمی گیرد، اما از درد طاقت فرسای آن می کاهد. • دندان درد: بهترین مسکن موقتی برای تسکین درد ناشی عفونت یا کشیدن دندان، شیاف دیکلوفناک۱۰۰ است. • زانودرد و درد مفاصل: کلیه عالئم آرتریت مفاصل شامل درد، التهاب، تورم و سفتی با شیاف دیکلوفناک به طور موقت قابل تسکین است. • گلودرد و تب: شیاف دیکلوفناک اث

## Chunking

In [ ]:

import re
from typing import List, Dict

_sentence_boundary_re = re.compile(r'(?<=[\.\!\؟\?؛\;\:\n])\s+')

def _split_sentences(text: str) -> List[str]:
    """
    Split text into sentences using the boundary regex.
    Trims whitespace from each sentence.
    """
    if not text:
        return []
    sents = [s.strip() for s in re.split(_sentence_boundary_re, text) if s and s.strip()]
    return sents

def chunk_word_based(pages: List[Dict],
                     chunk_size: int = 200,
                     overlap: int = 50) -> List[Dict]:
    """
    Word-based chunking per page.
    Args:
      pages: [{'page': int, 'text': str}, ...]
      chunk_size: number of words per chunk
      overlap: number of overlapping words between adjacent chunks
    Returns:
      list of chunks: {'id': str, 'page': int, 'start': int, 'end': int, 'text': str}
    """
    assert chunk_size > 0
    assert 0 <= overlap < chunk_size

    chunks = []
    for p in pages:
        page_num = p['page']
        words = p['text'].split()
        n = len(words)
        if n == 0:
            continue
        start = 0
        chunk_idx = 0
        step = chunk_size - overlap
        if step <= 0:
            step = chunk_size  # fallback to no overlap if invalid
        while start < n:
            end = min(start + chunk_size, n)
            chunk_text = " ".join(words[start:end])
            chunk_id = f"p{page_num}-w{chunk_idx}"
            chunks.append({
                'id': chunk_id,
                'page': page_num,
                'start_word': start,
                'end_word': end,
                'text': chunk_text
            })
            chunk_idx += 1
            start += step
    return chunks

def _chunk_sentence_fallback_word_split(sentence: str, max_words: int, page_num:int, base_id:str):
    """
    If a single sentence is longer than max_words, break it into word-based subchunks.
    """
    words = sentence.split()
    subchunks = []
    start = 0
    idx = 0
    while start < len(words):
        end = min(start + max_words, len(words))
        subchunks.append({
            'id': f"{base_id}-long-{idx}",
            'page': page_num,
            'start_word': start,
            'end_word': end,
            'text': " ".join(words[start:end])
        })
        idx += 1
        start += max_words
    return subchunks

def chunk_sentence_based(pages: List[Dict],
                         max_words_per_chunk: int = 200,
                         sentence_overlap: int = 1,
                         max_sentences_per_chunk: int = None) -> List[Dict]:
    """
    Sentence-based chunking per page.
    Args:
      pages: [{'page': int, 'text': str}, ...]
      max_words_per_chunk: approximate upper bound on words in a chunk
      sentence_overlap: number of sentences to repeat between adjacent chunks (helps context continuity)
      max_sentences_per_chunk: alternative bound on number of sentences (optional)
    Returns:
      list of chunks: {'id': str, 'page': int, 'sentences': [..], 'text': str}
    """
    chunks = []
    for p in pages:
        page_num = p['page']
        sents = _split_sentences(p['text'])
        if not sents:
            continue

        cur_sents = []
        cur_word_count = 0
        chunk_idx = 0
        i = 0
        while i < len(sents):
            sent = sents[i]
            sent_words = sent.split()
            # if one sentence alone exceeds the budget -> fallback split that sentence
            if len(sent_words) > max_words_per_chunk:
                # flush current accumulator first
                if cur_sents:
                    chunk_text = " ".join(cur_sents)
                    chunks.append({
                        'id': f"p{page_num}-s{chunk_idx}",
                        'page': page_num,
                        'sentences': cur_sents.copy(),
                        'text': chunk_text
                    })
                    chunk_idx += 1
                    # apply overlap logic to cur_sents for continuity
                    if sentence_overlap > 0:
                        cur_sents = cur_sents[-sentence_overlap:]
                    else:
                        cur_sents = []
                    cur_word_count = sum(len(s.split()) for s in cur_sents)
                # now split the long sentence into subchunks by words
                sub = _chunk_sentence_fallback_word_split(sent, max_words_per_chunk, page_num, f"p{page_num}-s{chunk_idx}")
                for sc in sub:
                    chunks.append(sc)
                    chunk_idx += 1
                i += 1
                continue

            # normal accumulation
            adding_wc = len(sent_words)
            if (cur_word_count + adding_wc <= max_words_per_chunk) and (max_sentences_per_chunk is None or len(cur_sents) + 1 <= max_sentences_per_chunk):
                cur_sents.append(sent)
                cur_word_count += adding_wc
                i += 1
            else:
                # flush current chunk
                if cur_sents:
                    chunk_text = " ".join(cur_sents)
                    chunks.append({
                        'id': f"p{page_num}-s{chunk_idx}",
                        'page': page_num,
                        'sentences': cur_sents.copy(),
                        'text': chunk_text
                    })
                    chunk_idx += 1
                # prepare overlap: keep last `sentence_overlap` sentences
                if sentence_overlap > 0:
                    cur_sents = cur_sents[-sentence_overlap:]
                    cur_word_count = sum(len(s.split()) for s in cur_sents)
                else:
                    cur_sents = []
                    cur_word_count = 0
        # flush leftover
        if cur_sents:
            chunk_text = " ".join(cur_sents)
            chunks.append({
                'id': f"p{page_num}-s{chunk_idx}",
                'page': page_num,
                'sentences': cur_sents.copy(),
                'text': chunk_text
            })
    return chunks




#### word_chunking

In [ ]:

pages, full_text = read_persian_pdf("/content/Drugs.pdf")
word_chunks = chunk_word_based(pages, chunk_size=200, overlap=50)

#
print("word chunks:", len(word_chunks))

print("sample word chunk text:\n", word_chunks[0]['text'][:300])


word chunks: 117
sample word chunk text:
 دیکلوفناک چیست و چه کاربردی دارد؟ دیکلوفناک یکی از پرمصرف ترین داروهای ضدالتهابی غیراستروئیدی است که فعالیت عوامل التهاب زا در بدن را مهار و با ایجاد التهاب و درد مقابله می کند. این دارو به: صورت دو ملح دیکلوفناک سدیم و پتاسیم در بازار موجود است دیکلوفناک سدیم با شروع اثر آهسته و طول اثر بیشتر و دیک


In [ ]:
chunks = word_chunks

In [ ]:
word_chunks[3]

{'id': 'p2-w0',
 'page': 2,
 'start_word': 0,
 'end_word': 200,
 'text': '۱. قرص، کپسول و شیاف دیکلوفناک پزشک برای تسکین درد طاقت فرسای بیماران مبتال به آرتریت یا دیگر شرایط دردناک جسمی، قرص، کپسول یا شیاف دیکلوفناک را تجویز می کند. مقدار دُز روزانه هر بیمار بین۷۵ تا۱۵۰ میلی گرم است و توسط پزشک تعیین می شود. مقدار مصرف معموال۱ تا۲ بار در روز با فاصله زمانی۱۰ تا۱۲ ساعت. میزان دقیق مصرف به وضعیت پزشکی و دیگر داروهای مصرفی بیمار بستگی دارد. نحوه مصرف قرص و کپسول قرص یا کپسول دیکلوفناک را با نوشیدنی شیر ببلعید تا کمتر موجب ناراحتی و تحریک معده شود. می توانید این قرص را یک ساعت قبل یا دو ساعت بعد از غذا با یک لیوان آب هم بخورید. فقط مراقب باشید که آن را در دهان نشکنید، نجوید یا له نکنید تا پوشش محافظ دارو از بین نرود. بعد از مصرف حداقل تا۱۰ دقیقه دراز نکشید. نحوه استفاده از شیاف شیاف را به آرامی به داخل مقعد فشار دهید و مطمئن شوید یک حدود۲ سانتی متر داخل شود. دست ها را بشویید و حدود۱۵ دقیقه بی حرکت بنشینید یا دراز بکشید تا دارو در بدن کامال ذوب و جذب شود. ۲. پماد دیکلوفناک پماد دیکلوفناک به

In [ ]:
# Extract text from word chunks
texts = [chunk['text'] for chunk in word_chunks]
print("Number of chunks:", len(texts))
print("Example chunk text:\n", texts[0][:200])


Number of chunks: 117
Example chunk text:
 دیکلوفناک چیست و چه کاربردی دارد؟ دیکلوفناک یکی از پرمصرف ترین داروهای ضدالتهابی غیراستروئیدی است که فعالیت عوامل التهاب زا در بدن را مهار و با ایجاد التهاب و درد مقابله می کند. این دارو به: صورت دو م


In [ ]:
type(texts)

list

In [ ]:
texts[0]

'دیکلوفناک چیست و چه کاربردی دارد؟ دیکلوفناک یکی از پرمصرف ترین داروهای ضدالتهابی غیراستروئیدی است که فعالیت عوامل التهاب زا در بدن را مهار و با ایجاد التهاب و درد مقابله می کند. این دارو به: صورت دو ملح دیکلوفناک سدیم و پتاسیم در بازار موجود است دیکلوفناک سدیم با شروع اثر آهسته و طول اثر بیشتر و دیکلوفناک پتاسیم با شروع اثر سریع و طول اثر کوتاه. این دارو در اشکال دارویی قرص، کپسول، پماد، شیاف و آمپول برای استفاده خوراکی یا موضعی در تمام داروخانه ها یافت می شود و به محض استفاده، دردهای حاد و ناگهانی نیازمند تسکین سریع درد را آرام می سازد. کاربرد شیاف دیکلوفناک از کاربردهای شیاف مسکن دیکلوفناک می توان به این موارد اشاره کرد: • تسکین سردرد و میگرن: این شیاف جلوی حمالت میگرن را نمی گیرد، اما از درد طاقت فرسای آن می کاهد. • دندان درد: بهترین مسکن موقتی برای تسکین درد ناشی عفونت یا کشیدن دندان، شیاف دیکلوفناک۱۰۰ است. • زانودرد و درد مفاصل: کلیه عالئم آرتریت مفاصل شامل درد، التهاب، تورم و سفتی با شیاف دیکلوفناک به طور موقت قابل تسکین است. • گلودرد و تب: شیاف دیکلوفناک اثر ضد درد و تب بری دار

#### Sentence_chunking

In [ ]:
##sentence-based
sent_chunks = chunk_sentence_based(pages, max_words_per_chunk=200, sentence_overlap=1)
print("sentence chunks:", len(sent_chunks))
print("sample sentence chunk text:\n", sent_chunks[0]['text'][:300])

sentence chunks: 98
sample sentence chunk text:
 دیکلوفناک چیست و چه کاربردی دارد؟ دیکلوفناک یکی از پرمصرف ترین داروهای ضدالتهابی غیراستروئیدی است که فعالیت عوامل التهاب زا در بدن را مهار و با ایجاد التهاب و درد مقابله می کند. این دارو به: صورت دو ملح دیکلوفناک سدیم و پتاسیم در بازار موجود است دیکلوفناک سدیم با شروع اثر آهسته و طول اثر بیشتر و دیک


In [ ]:
chunks = sent_chunks

In [ ]:
sent_chunks[0]

{'id': 'p1-s0',
 'page': 1,
 'sentences': ['دیکلوفناک چیست و چه کاربردی دارد؟',
  'دیکلوفناک یکی از پرمصرف ترین داروهای ضدالتهابی غیراستروئیدی است که فعالیت عوامل التهاب زا در بدن را مهار و با ایجاد التهاب و درد مقابله می کند.',
  'این دارو به:',
  'صورت دو ملح دیکلوفناک سدیم و پتاسیم در بازار موجود است دیکلوفناک سدیم با شروع اثر آهسته و طول اثر بیشتر و دیکلوفناک پتاسیم با شروع اثر سریع و طول اثر کوتاه.',
  'این دارو در اشکال دارویی قرص، کپسول، پماد، شیاف و آمپول برای استفاده خوراکی یا موضعی در تمام داروخانه ها یافت می شود و به محض استفاده، دردهای حاد و ناگهانی نیازمند تسکین سریع درد را آرام می سازد.',
  'کاربرد شیاف دیکلوفناک از کاربردهای شیاف مسکن دیکلوفناک می توان به این موارد اشاره کرد:',
  '• تسکین سردرد و میگرن:',
  'این شیاف جلوی حمالت میگرن را نمی گیرد، اما از درد طاقت فرسای آن می کاهد.',
  '• دندان درد:',
  'بهترین مسکن موقتی برای تسکین درد ناشی عفونت یا کشیدن دندان، شیاف دیکلوفناک۱۰۰ است.',
  '• زانودرد و درد مفاصل:',
  'کلیه عالئم آرتریت مفاصل شامل درد، التهاب، تورم و سفتی ب

In [ ]:
# Extract text from word chunks
texts = [chunk['text'] for chunk in sent_chunks]
print("Number of chunks:", len(texts))
print("Example chunk text:\n", texts[0])

Number of chunks: 98
Example chunk text:
 دیکلوفناک چیست و چه کاربردی دارد؟ دیکلوفناک یکی از پرمصرف ترین داروهای ضدالتهابی غیراستروئیدی است که فعالیت عوامل التهاب زا در بدن را مهار و با ایجاد التهاب و درد مقابله می کند. این دارو به: صورت دو ملح دیکلوفناک سدیم و پتاسیم در بازار موجود است دیکلوفناک سدیم با شروع اثر آهسته و طول اثر بیشتر و دیکلوفناک پتاسیم با شروع اثر سریع و طول اثر کوتاه. این دارو در اشکال دارویی قرص، کپسول، پماد، شیاف و آمپول برای استفاده خوراکی یا موضعی در تمام داروخانه ها یافت می شود و به محض استفاده، دردهای حاد و ناگهانی نیازمند تسکین سریع درد را آرام می سازد. کاربرد شیاف دیکلوفناک از کاربردهای شیاف مسکن دیکلوفناک می توان به این موارد اشاره کرد: • تسکین سردرد و میگرن: این شیاف جلوی حمالت میگرن را نمی گیرد، اما از درد طاقت فرسای آن می کاهد. • دندان درد: بهترین مسکن موقتی برای تسکین درد ناشی عفونت یا کشیدن دندان، شیاف دیکلوفناک۱۰۰ است. • زانودرد و درد مفاصل: کلیه عالئم آرتریت مفاصل شامل درد، التهاب، تورم و سفتی با شیاف دیکلوفناک به طور موقت قابل تسکین است. • گلودرد و ت

In [ ]:
chunks[0]

{'id': 'p1-s0',
 'page': 1,
 'sentences': ['دیکلوفناک چیست و چه کاربردی دارد؟',
  'دیکلوفناک یکی از پرمصرف ترین داروهای ضدالتهابی غیراستروئیدی است که فعالیت عوامل التهاب زا در بدن را مهار و با ایجاد التهاب و درد مقابله می کند.',
  'این دارو به:',
  'صورت دو ملح دیکلوفناک سدیم و پتاسیم در بازار موجود است دیکلوفناک سدیم با شروع اثر آهسته و طول اثر بیشتر و دیکلوفناک پتاسیم با شروع اثر سریع و طول اثر کوتاه.',
  'این دارو در اشکال دارویی قرص، کپسول، پماد، شیاف و آمپول برای استفاده خوراکی یا موضعی در تمام داروخانه ها یافت می شود و به محض استفاده، دردهای حاد و ناگهانی نیازمند تسکین سریع درد را آرام می سازد.',
  'کاربرد شیاف دیکلوفناک از کاربردهای شیاف مسکن دیکلوفناک می توان به این موارد اشاره کرد:',
  '• تسکین سردرد و میگرن:',
  'این شیاف جلوی حمالت میگرن را نمی گیرد، اما از درد طاقت فرسای آن می کاهد.',
  '• دندان درد:',
  'بهترین مسکن موقتی برای تسکین درد ناشی عفونت یا کشیدن دندان، شیاف دیکلوفناک۱۰۰ است.',
  '• زانودرد و درد مفاصل:',
  'کلیه عالئم آرتریت مفاصل شامل درد، التهاب، تورم و سفتی ب

##Embedding model

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Generate embeddings for all chunks
embeddings = model.encode(
    texts,
    batch_size=32,      # adjust depending on your GPU memory
    show_progress_bar=True
)

print("Shape of embeddings:", embeddings.shape)  # e.g., (num_chunks, 384)


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Shape of embeddings: (98, 384)


In [ ]:
# Combine embeddings with chunk info for easy storage
chunked_data_with_embeddings = []

for i, chunk in enumerate(chunks):
    chunk_copy = chunk.copy()      # avoid modifying original
    chunk_copy['embedding'] = embeddings[i]  # add embedding vector
    chunked_data_with_embeddings.append(chunk_copy)

# Check
print("Sample chunk with embedding:")
print(chunked_data_with_embeddings[0]['id'])
print("Embedding vector length:", len(chunked_data_with_embeddings[0]['embedding']))


Sample chunk with embedding:
p1-s0
Embedding vector length: 384


In [ ]:
chunked_data_with_embeddings[0]

{'id': 'p1-s0',
 'page': 1,
 'sentences': ['دیکلوفناک چیست و چه کاربردی دارد؟',
  'دیکلوفناک یکی از پرمصرف ترین داروهای ضدالتهابی غیراستروئیدی است که فعالیت عوامل التهاب زا در بدن را مهار و با ایجاد التهاب و درد مقابله می کند.',
  'این دارو به:',
  'صورت دو ملح دیکلوفناک سدیم و پتاسیم در بازار موجود است دیکلوفناک سدیم با شروع اثر آهسته و طول اثر بیشتر و دیکلوفناک پتاسیم با شروع اثر سریع و طول اثر کوتاه.',
  'این دارو در اشکال دارویی قرص، کپسول، پماد، شیاف و آمپول برای استفاده خوراکی یا موضعی در تمام داروخانه ها یافت می شود و به محض استفاده، دردهای حاد و ناگهانی نیازمند تسکین سریع درد را آرام می سازد.',
  'کاربرد شیاف دیکلوفناک از کاربردهای شیاف مسکن دیکلوفناک می توان به این موارد اشاره کرد:',
  '• تسکین سردرد و میگرن:',
  'این شیاف جلوی حمالت میگرن را نمی گیرد، اما از درد طاقت فرسای آن می کاهد.',
  '• دندان درد:',
  'بهترین مسکن موقتی برای تسکین درد ناشی عفونت یا کشیدن دندان، شیاف دیکلوفناک۱۰۰ است.',
  '• زانودرد و درد مفاصل:',
  'کلیه عالئم آرتریت مفاصل شامل درد، التهاب، تورم و سفتی ب

## Vector db

In [ ]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 18.6 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pickle


# 3. Create FAISS index
dimension = embeddings.shape[1]  # 384 for MiniLM
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# 4. Store index and metadata
faiss.write_index(index, "word_chunks_index.faiss")

with open("chunks_metadata.pkl", "wb") as f:
    pickle.dump(word_chunks, f)

print(f"Saved {len(word_chunks)} chunks with embeddings.")


Saved 117 chunks with embeddings.


In [ ]:
import faiss
import numpy as np
import pickle

# 1. Build FAISS index
dim = len(chunked_data_with_embeddings[0]['embedding'])
index = faiss.IndexFlatL2(dim)

# Add embeddings
embeddings = np.array([c['embedding'] for c in chunked_data_with_embeddings], dtype="float32")
index.add(embeddings)

# 2. Save metadata (id, text, page, etc.)
metadata = [
    {k: v for k, v in c.items() if k != "embedding"}
    for c in chunked_data_with_embeddings
]

# Save both
faiss.write_index(index, "word_chunk_faiss_index.bin")
with open("metadata.pkl", "wb") as f:
    pickle.dump(metadata, f)


## Load it

In [ ]:
import faiss
import pickle
import numpy as np

# ---- Load FAISS index and metadata ----
index = faiss.read_index("word_chunk_faiss_index.bin")

with open("metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

print(f"Index has {index.ntotal} vectors")
print(f"Metadata has {len(metadata)} entries")

# ---- Example: make a query ----
# Let's say you have an embedding model (e.g., sentence-transformers)
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

query = "کاربرد دیکلوفناک چیست؟"
query_embedding = embedder.encode([query], convert_to_numpy=True).astype("float32")

# ---- Search in FAISS ----
k = 5  # number of nearest neighbors
distances, indices = index.search(query_embedding, k)

print("\nTop results:")
for rank, (idx, dist) in enumerate(zip(indices[0], distances[0]), start=1):
    item = metadata[idx]
    print(f"{rank}. (distance={dist:.4f}) Page={item['page']} → {item['text'][:200]}...")


Index has 117 vectors
Metadata has 117 entries


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


Top results:
1. (distance=4.6722) Page=27 → آمپول کلرفنیرامین برای چیست؟ آمپول کلرفنیرامین برای درمان مشکالت التهابی و حساسیتی مثل ورم و کهیر و بهبود عالئم خارش، درد، آبریزش بینی، اشک و خارش چشم و غیره تجویز می شود. این آمپول فقط تحت نظر پزشک ت...
2. (distance=4.6821) Page=11 → سوءمصرف کلیندامایسین در صورت سوء مصرف کلیندامایسین با عالئمی مثل غش ( یا مشکل تنفسی، با اورژانس۱۱۵ )تماس بگیرید. عوارض جانبی کلیندامایسین و هشدارها کلیندامایسین با توجه به شرایط شما تجویز شده است. به ...
3. (distance=4.7723) Page=1 → دیکلوفناک چیست و چه کاربردی دارد؟ دیکلوفناک یکی از پرمصرف ترین داروهای ضدالتهابی غیراستروئیدی است که فعالیت عوامل التهاب زا در بدن را مهار و با ایجاد التهاب و درد مقابله می کند. این دارو به: صورت دو م...
4. (distance=5.3938) Page=18 → غیردارویی یا مصرف سایر داروهای مربوط به درمان درد با پزشک مشورت کنید. همچنین بخش هشدار را مالحظه نمایید. نحوه مصرف ایندومتاسین خوراکی برای مصرف ایندومتاسین به این نکات حتماً توجه کنید: • قبل از شروع م...
5. (distance=5.4535) Page=19 → وقت های پزشکی و 

In [ ]:
metadata[0]

{'id': 'p1-w0',
 'page': 1,
 'start_word': 0,
 'end_word': 200,
 'text': 'دیکلوفناک چیست و چه کاربردی دارد؟ دیکلوفناک یکی از پرمصرف ترین داروهای ضدالتهابی غیراستروئیدی است که فعالیت عوامل التهاب زا در بدن را مهار و با ایجاد التهاب و درد مقابله می کند. این دارو به: صورت دو ملح دیکلوفناک سدیم و پتاسیم در بازار موجود است دیکلوفناک سدیم با شروع اثر آهسته و طول اثر بیشتر و دیکلوفناک پتاسیم با شروع اثر سریع و طول اثر کوتاه. این دارو در اشکال دارویی قرص، کپسول، پماد، شیاف و آمپول برای استفاده خوراکی یا موضعی در تمام داروخانه ها یافت می شود و به محض استفاده، دردهای حاد و ناگهانی نیازمند تسکین سریع درد را آرام می سازد. کاربرد شیاف دیکلوفناک از کاربردهای شیاف مسکن دیکلوفناک می توان به این موارد اشاره کرد: • تسکین سردرد و میگرن: این شیاف جلوی حمالت میگرن را نمی گیرد، اما از درد طاقت فرسای آن می کاهد. • دندان درد: بهترین مسکن موقتی برای تسکین درد ناشی عفونت یا کشیدن دندان، شیاف دیکلوفناک۱۰۰ است. • زانودرد و درد مفاصل: کلیه عالئم آرتریت مفاصل شامل درد، التهاب، تورم و سفتی با شیاف دیکلوفناک به طور م

In [ ]:
distances

array([[4.6722465, 4.6821404, 4.772269 , 5.3937635, 5.453493 ]],
      dtype=float32)

In [ ]:
indices

array([[65, 28,  0, 43, 46]])

## working with LLM

INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found exist

In [ ]:
%%writefile .env
GOOGLE_API_KEY=Your_API_KEY

Writing .env


In [ ]:
import os
from langchain_google_genai import GoogleGenerativeAI


In [ ]:
import os
from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv



load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

llm = GoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=GOOGLE_API_KEY, temperature= 0)


In [ ]:
llm("Hi")

'Hi there! How can I help you today?'

### TF-IDF

In [ ]:
!pip install --quiet scikit-learn

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pickle
import os

# ------------- Prepare corpus -------------
docs = [c['text'] for c in chunked_data_with_embeddings]
ids  = [c['id']   for c in chunked_data_with_embeddings]
metas = [{k: v for k, v in c.items() if k != 'embedding'} for c in chunked_data_with_embeddings]

print("Number of chunks:", len(docs))

# ------------- Build TF-IDF vectorizer -------------
# You can tune token_pattern/ngram_range/max_features for Persian specifics.
# token_pattern uses \w which covers Unicode letters including Persian.
vectorizer = TfidfVectorizer(
    lowercase=True,
    analyzer='word',
    token_pattern=r"(?u)\b\w+\b",   # basic unicode word tokenization
    ngram_range=(1,2),              # unigrams + bigrams often helpful
    max_features=None               # or set e.g. 50000 if you want to limit memory
)

tfidf_matrix = vectorizer.fit_transform(docs)  # shape: (n_docs, n_features)
print("TF-IDF matrix shape:", tfidf_matrix.shape)

# Optional: persist vectorizer + matrix + metadata for later use
os.makedirs("tfidf_store", exist_ok=True)
with open("tfidf_store/vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)
# Save sparse matrix efficiently
from scipy import sparse
sparse.save_npz("tfidf_store/tfidf_matrix.npz", tfidf_matrix)
with open("tfidf_store/metadata.pkl", "wb") as f:
    pickle.dump(metas, f)

# ------------- Query function -------------
def tfidf_retrieve(query: str, top_k: int = 5, return_scores: bool=True):
    """
    Returns list of (metadata, score) tuples (sorted desc by score).
    score = cosine similarity between query and doc tf-idf vectors.
    """
    q_vec = vectorizer.transform([query])  # (1, n_features)
    # cosine_similarity handles sparse inputs
    sims = cosine_similarity(q_vec, tfidf_matrix).flatten()  # shape (n_docs,)
    top_idx = np.argsort(-sims)[:top_k]
    results = []
    for i in top_idx:
        results.append((metas[i], float(sims[i])))
    if return_scores:
        return results
    else:
        return [r[0] for r in results]

# ------------- Example usage -------------
query = "کاربرد دیکلوفناک چیست؟"
results = tfidf_retrieve(query, top_k=6)
print("Top results for query:", query)
for rank, (meta, score) in enumerate(results, start=1):
    txt = meta.get('text','')[:250].replace("\n"," ")
    print(f"{rank}. id={meta.get('id')} page={meta.get('page')}  score={score:.4f}")
    print("   ", txt, "...\n")


Number of chunks: 98
TF-IDF matrix shape: (98, 9517)
Top results for query: کاربرد دیکلوفناک چیست؟
1. id=p1-s0 page=1  score=0.1695
    دیکلوفناک چیست و چه کاربردی دارد؟ دیکلوفناک یکی از پرمصرف ترین داروهای ضدالتهابی غیراستروئیدی است که فعالیت عوامل التهاب زا در بدن را مهار و با ایجاد التهاب و درد مقابله می کند. این دارو به: صورت دو ملح دیکلوفناک سدیم و پتاسیم در بازار موجود است دیکل ...

2. id=p6-s1 page=6  score=0.1081
    داروی جایگزین دیکلوفناک چنانچه به دنبال یک جایگزین مؤثر برای داروی دیکلوفناک می، گردید بعد از مشاوره با پزشک ترجیحا سراغ داروهای ضدالتهابی غیراستروئیدی مثل ایبوپروفن و ناپروکسن، بروید به ویژه اگر از دردهای مزمن رنج می برید و دارویی برای استفاده طوالن ...

3. id=p5-s1 page=5  score=0.1029
    • اگر فشار خون باال یا سابقه احتباس مایعات دارید، با مصرف دیکلوفناک حجم کار قلب را بیشتر و سخت تر می کنید. • این دارو بعد از هفته۳۰ ام بارداری (در بعضی از منابع حتی هفته۲۰ام به بعد) در دسته داروهای «D» قرار می گیرد یعنی مصرف آن می تواند برای جنین خطر ...

4. id=p5-s0 page=5  sc

### BM-25

In [ ]:

!pip install --quiet rank_bm25


In [ ]:

from rank_bm25 import BM25Okapi
import re
import pickle
import os
import numpy as np

# --- simple normalize/tokenize helpers (reuse your normalizer if you have one) ---
def simple_normalize(text: str) -> str:
    # minimal normalization: remove BOM/zero-width, map characters, collapse spaces
    text = text.replace('\ufeff', ' ')
    text = re.sub(r'[\u200c\u200b\u200d\u200e\u200f]+', ' ', text)
    text = text.replace('ي', 'ی').replace('ك', 'ک')
    text = re.sub(r'[\x00-\x1f]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

_token_re = re.compile(r"(?u)\b\w+\b")  # Unicode-aware word tokenizer

def tokenize(text: str):
    text = simple_normalize(text)
    # lowercase is not always relevant for Persian but harmless
    text = text.lower()
    return _token_re.findall(text)

# --- Build BM25 index ---
docs = [c['text'] for c in chunked_data_with_embeddings]
metas = [{k:v for k,v in c.items() if k != 'embedding'} for c in chunked_data_with_embeddings]

# Tokenize corpus (list of list[str])
tokenized_corpus = [tokenize(doc) for doc in docs]

# Initialize BM25 (k1 and b are tunable; defaults are common)
bm25 = BM25Okapi(tokenized_corpus, k1=1.5, b=0.75)

# Persist BM25 and metadata
os.makedirs("bm25_store", exist_ok=True)
with open("bm25_store/bm25.pkl", "wb") as f:
    pickle.dump(bm25, f)
with open("bm25_store/tokenized_corpus.pkl", "wb") as f:
    pickle.dump(tokenized_corpus, f)
with open("bm25_store/metadata.pkl", "wb") as f:
    pickle.dump(metas, f)

print("Built BM25 index for", len(docs), "chunks")

# --- BM25 query function ---
def bm25_retrieve(query: str, top_k: int = 5):
    q_tokens = tokenize(query)
    # get scores (higher = more relevant)
    scores = bm25.get_scores(q_tokens)  # numpy array or list
    top_idx = np.argsort(-scores)[:top_k]
    results = []
    for i in top_idx:
        results.append((metas[i], float(scores[i])))
    return results

# --- Example usage ---
query = "کاربرد دیکلوفناک چیست؟"
results = bm25_retrieve(query, top_k=6)
print("Top BM25 results for query:", query)
for rank, (meta, score) in enumerate(results, start=1):
    txt = meta.get('text','')[:240].replace("\n"," ")
    print(f"{rank}. id={meta.get('id')} page={meta.get('page')} score={score:.4f}")
    print("    ", txt, "...\n")


Built BM25 index for 98 chunks
Top BM25 results for query: کاربرد دیکلوفناک چیست؟
1. id=p1-s0 page=1 score=7.7786
     دیکلوفناک چیست و چه کاربردی دارد؟ دیکلوفناک یکی از پرمصرف ترین داروهای ضدالتهابی غیراستروئیدی است که فعالیت عوامل التهاب زا در بدن را مهار و با ایجاد التهاب و درد مقابله می کند. این دارو به: صورت دو ملح دیکلوفناک سدیم و پتاسیم در بازار موجو ...

2. id=p40-s0 page=40 score=5.5659
     آسپرین چیست و چه کاربردی دارد؟ آسپرین (Aspirin) یک عالمت تجاری متعلق به شرکت داروسازی آلمانی بایر است با اسم عمومی استیل سالیسیلیک اسید یا آ اس آ (Asa) است. این دارو یک داروی ضدالتهابی غیراستروئیدی رایج و اولین رده از این داروها است که برای ...

3. id=p1-s1 page=1 score=5.1596
     • گلودرد و تب: شیاف دیکلوفناک اثر ضد درد و تب بری دارد و دُزهای مناسب آن در کودکان و بزرگ ساالن قابل استفاده است. • درد قاعدگی: یکی از موارد مصرف شیاف دیکلوفناک تسکین درد قاعدگی است و ظرف۱۰ دقیقه این درد را آرام می کند. کاربرد آمپول دیکلوفن ...

4. id=p7-s0 page=7 score=4.2049
     قرص کلیندامایسین چیست؟ کلیندام

## Fine-tuning


#### Generate dataset

In [ ]:
# Paste into Colab
import json
import re
import uuid
import os
from tqdm import tqdm
from typing import Callable, Dict, List, Tuple, Any, Optional
from collections import OrderedDict

# ---------------- Helper utilities ----------------
def _strip_numbering(line: str) -> str:
    return re.sub(r'^\s*\d+[\)\.\-:\s]*', '', line).strip()

def _normalize_whitespace(s: str) -> str:
    return re.sub(r'\s+', ' ', s).strip()

def _tokenize_simple(s: str) -> List[str]:
    # simple Unicode word tokens (works okay for Persian)
    return re.findall(r"(?u)\b\w+\b", s)

# ---------------- Main generator function ----------------
def generate_synthetic_queries(
    chunked_data: List[Dict[str, Any]],
    llm_fn: Callable[[str], str],
    num_questions_per_chunk: int = 2,
    out_dir: str = "./data",
    train_frac: float = 0.8,
    min_question_words: int = 5,
    max_question_words: int = 25,
    require_token_overlap: bool = True,
    overlap_token_threshold: int = 1,
    dedupe: bool = True,
    verbose: bool = True,
    test_mode_chunks: Optional[int] = None,  # set small int for quick dry-run
) -> Tuple[Dict[str,str], Dict[str, List[str]]]:
    """
    Generate synthetic (question -> relevant_chunk_id) pairs.
    - chunked_data: list of {'id','page','start_word','end_word','text', ...}
    - llm_fn: callable(prompt: str) -> str (should return LLM text output)
    - saves train/val corpus + queries + relevant_docs in out_dir
    Returns (all_queries_dict, all_relevant_docs_dict)
    """
    os.makedirs(out_dir, exist_ok=True)
    # Build corpus mapping chunk_id->text
    corpus = OrderedDict()
    for c in chunked_data:
        cid = c['id']
        corpus[cid] = _simple_persian_normalize(c['text'])
    # Optionally limit for quick test
    all_chunk_ids = list(corpus.keys())
    if test_mode_chunks is not None:
        all_chunk_ids = all_chunk_ids[:test_mode_chunks]

    # Split train/val by simple fraction (contiguous split is fine for small corpora)
    n = len(all_chunk_ids)
    split_idx = int(n * train_frac)
    train_ids = all_chunk_ids[:split_idx]
    val_ids   = all_chunk_ids[split_idx:]

    if verbose:
        print(f"Total chunks considered: {n}, train: {len(train_ids)}, val: {len(val_ids)}")

    # prompt template (Persian) - uses {N} and {context_str}
    prompt_template = (
        "متن زیر را بخوان و بر اساس آن دقیقاً {N} پرسش کوتاه و قابل‌پاسخ تولید کن.\n\n"
        "قوانین:\n"
        "1) پرسش‌ها باید مستقیماً از متن قابل پاسخ باشند (بدون نیاز به دانش خارج از متن).\n"
        "2) پرسش‌ها را شماره‌گذاری کن و فقط متن پرسش را برگردان (بدون پاسخ یا توضیح).\n"
        "3) از پرسش‌های بسیار کلی پرهیز کن؛ پرسش‌ها باید مشخص و مرتبط با متن باشند.\n"
        "4) هر پرسش بین {MIN} تا {MAX} کلمه باشد.\n\n"
        "فرمت خروجی (هر خط یکی):\n"
        "1) <پرسش اول>\n"
        "2) <پرسش دوم>\n"
        "...\n\n"
        "متن:\n\"\"\"\n{context_str}\n\"\"\"\n"
    )

    def _build_prompt(chunk_text: str, N: int):
        return prompt_template.format(N=N, context_str=chunk_text, MIN=min_question_words, MAX=max_question_words)

    # Function to parse LLM response: numbered lines -> list of strings
    def _parse_numbered_lines(text: str) -> List[str]:
        lines = text.splitlines()
        questions = []
        for ln in lines:
            ln = ln.strip()
            if not ln:
                continue
            # allow "1)", "1.", "1 -", "1 )" prefixes
            q = _strip_numbering(ln)
            if q:
                questions.append(q)
        # if parsing produced zero lines, try splitting on double newline
        if len(questions) == 0:
            parts = text.split("\n\n")
            for p in parts:
                p = p.strip()
                if p:
                    q = _strip_numbering(p)
                    if q:
                        questions.append(q)
        return questions

    # Stores
    train_queries = {}
    train_relevant_docs = {}
    val_queries = {}
    val_relevant_docs = {}
    global_queries = {}
    global_relevant = {}

    seen_questions = set()

    def _accept_question(q: str, chunk_text: str) -> bool:
        q_norm = _simple_persian_normalize(q)
        # length filter by words
        wcount = len(_tokenize_simple(q_norm))
        if wcount < min_question_words or wcount > max_question_words:
            return False
        # avoid tiny uninformative questions
        if len(q_norm) < 4:
            return False
        # token overlap check
        if require_token_overlap:
            q_tokens = set(_tokenize_simple(q_norm))
            chunk_tokens = set(_tokenize_simple(chunk_text))
            if len(q_tokens & chunk_tokens) < overlap_token_threshold:
                return False
        return True

    # Main loop for generating questions
    def _process_chunk_ids(chunk_ids, out_queries, out_relevant, label):
        for cid in tqdm(chunk_ids, desc=f"Generating ({label})"):
            text = corpus[cid]
            prompt = _build_prompt(text, num_questions_per_chunk)
            try:
                response = llm_fn(prompt)
            except Exception as e:
                print(f"[WARN] LLM call failed for {cid}: {e}")
                continue
            # raw parse
            questions = _parse_numbered_lines(response)
            # if no questions, try to use whole response as single question (fallback)
            if len(questions) == 0 and response.strip():
                questions = [response.strip()]

            # iterate and validate
            for q in questions:
                q = _simple_persian_normalize(q)
                if dedupe and q in seen_questions:
                    continue
                if not _accept_question(q, text):
                    continue
                qid = str(uuid.uuid4())
                out_queries[qid] = q
                out_relevant[qid] = [cid]
                global_queries[qid] = q
                global_relevant[qid] = [cid]
                seen_questions.add(q)

    # Run for train and val
    _process_chunk_ids(train_ids, train_queries, train_relevant_docs, "train")
    _process_chunk_ids(val_ids,   val_queries,   val_relevant_docs,   "val")

    # Save files
    def _save_json(obj, fname):
        path = os.path.join(out_dir, fname)
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(obj, f, ensure_ascii=False, indent=2)
        if verbose:
            print("Saved:", path)

    # Save corpora and query/relevant maps
    train_corpus = {cid: corpus[cid] for cid in train_ids}
    val_corpus   = {cid: corpus[cid] for cid in val_ids}

    _save_json(train_corpus, "train_corpus.json")
    _save_json(val_corpus, "val_corpus.json")
    _save_json(train_queries, "train_queries.json")
    _save_json(train_relevant_docs, "train_relevant_docs.json")
    _save_json(val_queries, "val_queries.json")
    _save_json(val_relevant_docs, "val_relevant_docs.json")

    # Combined dataset files
    _save_json({"queries": train_queries, "corpus": train_corpus, "relevant_docs": train_relevant_docs}, "train_dataset.json")
    _save_json({"queries": val_queries,   "corpus": val_corpus,   "relevant_docs": val_relevant_docs},   "val_dataset.json")

    if verbose:
        print(f"Generated total questions: {len(global_queries)} (train={len(train_queries)}, val={len(val_queries)})")
    return global_queries, global_relevant



In [ ]:
# ---------------- Example usage ----------------
# Suppose you previously created `chunked_data_with_embeddings` in your Colab.
# And you have `llm = GoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=..., temperature=0)`
# Create a wrapper llm_fn that returns a plain text string from your LLM object.
#
# Example wrappers (choose the one that matches your client API):
#
# 1) If calling llm with llm(prompt) works and returns plain str:
llm_fn = lambda prompt: llm(prompt)
#
# 2) If your library uses .generate and returns object (change extraction as required):
# llm_fn = lambda prompt: llm.generate([prompt]).generations[0][0].text
#
# 3) If your library has .invoke or .complete that returns text:
# llm_fn = lambda prompt: llm.invoke(prompt)   # or .complete(prompt)
#
# Test with a tiny subset first:
test_queries, test_relevants = generate_synthetic_queries(chunked_data_with_embeddings, llm_fn, num_questions_per_chunk=4, out_dir="./data", test_mode_chunks=None )
#
# When you are satisfied, run with test_mode_chunks=None.


Total chunks considered: 98, train: 78, val: 20


Generating (train):  27%|██▋       | 21/78 [01:57<04:44,  4.99s/it]WARNING:langchain_core.language_models.llms:Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 42
}
].
Generating (train):  53%|█████▎    | 41/78 [03:58<03:11,  5.18s/it]WARNING:langchain_core

Saved: ./data/train_corpus.json
Saved: ./data/val_corpus.json
Saved: ./data/train_queries.json
Saved: ./data/train_relevant_docs.json
Saved: ./data/val_queries.json
Saved: ./data/val_relevant_docs.json
Saved: ./data/train_dataset.json
Saved: ./data/val_dataset.json
Generated total questions: 388 (train=309, val=79)


In [ ]:
len(chunked_data_with_embeddings)

98

In [ ]:
import json

# Look at train dataset
with open("./data/train_dataset.json", "r") as f:
    train_data = json.load(f)

print("Number of training examples:", len(train_data))
print("Keys:", train_data.keys())



# Same for validation
with open("./data/val_dataset.json", "r") as f:
    val_data = json.load(f)

print("\nNumber of validation examples:", len(val_data))
print("keys:", val_data.keys())

Number of training examples: 3
Keys: dict_keys(['queries', 'corpus', 'relevant_docs'])

Number of validation examples: 3
keys: dict_keys(['queries', 'corpus', 'relevant_docs'])


In [ ]:
train_data

{'queries': {'7645a669-81df-4cd7-a2b2-ca503100b1e0': 'دیکلوفناک چگونه با التهاب و درد مقابله می کند؟',
  '0df0cc7c-0dbc-4d02-a4c7-3d9a6c320502': 'دیکلوفناک به صورت چند ملح در بازار موجود است؟',
  'b7f349cb-7927-40c0-ba47-857906fc2ee1': 'تفاوت دیکلوفناک سدیم و پتاسیم در چیست؟',
  '86c5b2ec-aeab-466c-8875-17c37f52e950': 'شیاف دیکلوفناک چه علائم آرتریت مفاصل را تسکین می دهد؟',
  'a6601503-7266-458d-9d07-85681526d987': 'شیاف دیکلوفناک برای تسکین درد قاعدگی چقدر زمان می برد؟',
  'b87c36b2-91e3-47f5-a63a-3911868feece': 'آمپول دیکلوفناک چه علائمی را در بیماری آرتروز آرام می کند؟',
  'd0fffdb7-e7ca-4c00-a046-a8919821aaa9': 'دیکلوفناک تولید کدام آنزیم التهاب ساز را مهار می کند؟',
  'be587b32-cdab-49fa-b09e-ad289d759d6b': 'دُز بالای آمپول دیکلوفناک به عنوان چه نوع مسکنی شناخته می شود؟',
  '6a4a27f8-8bf7-41c1-b23f-2923513b4202': 'با کاهش میزان مواد شیمیایی التهابی در بدن چه اتفاقی می افتد؟',
  '2da0f6d4-ddf8-4a58-ab01-afcc45b53032': 'پزشک معالج بعد از تجویز دیکلوفناک چه چیزی به بیمار می دهد؟',
  

In [ ]:
val_data

{'queries': {'a3c7af07-ad33-4fcb-9abe-23225dc2b1cc': 'آسپرین برای درد کدام بیماری های طولانی مدت تجویز می شود؟',
  '91e89911-2087-46b1-be63-bcdd0f3209fa': 'آسپرین با دُز کم چگونه خطر بیماری قلبی عروقی را کاهش می دهد؟',
  '28e7fad8-5756-49ce-8145-0078a7850ebb': 'در چه شرایطی پزشک مصرف روزانه دُز پایین آسپرین را پیشنهاد می دهد؟',
  'b5c62c36-5852-4d31-9f5b-17e0d13b8303': 'برای درد شدید، پزشک ممکن است آسپرین را با چه داروهایی تجویز کند؟',
  'da6f5e0d-da0c-4a4d-900f-5132cb22f923': 'در افرادی که مشکالت ندارند، خطر مصرف طولانی مدت آسپرین چگونه است؟',
  '7c21f49f-d8f3-4b92-8eee-4b9502bc7a98': 'افراد بین ۵۰ تا ۵۹ سال برای جلوگیری از چه بیماری می توانند آسپرین مصرف کنند؟',
  '12907f60-adc3-40f4-80ed-e2379aaf5ead': 'خطر ۱۰ ساله بیماری های قلبی عروقی در بزرگسالان توصیه شده چقدر باید باشد؟',
  '234cb9c4-62de-429d-8e74-f98aefb73985': 'آسپرین برای درمان چه بیماری در متن ذکر شده است؟',
  '40d50c59-2c97-4460-b419-79d96d949d75': 'آسپرین برای درمان چه نوع سکته مغزی تجویز نمی شود؟',
  'd137f677-efc8-4d21

#### Saving dataset

In [ ]:
import shutil
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:


# Path to dataset and zip file
dataset_path = '/content/data'
zip_path = "/content/drive/MyDrive/Colab Notebooks/AI_task/RAG/dataset_sentence"

# Zip the dataset folder
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', dataset_path)

print("Dataset zipped and saved at:", zip_path)


Dataset zipped and saved at: /content/drive/MyDrive/Colab Notebooks/AI_task/RAG/dataset_sentence


#### fine-tuning

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from torch.utils.data import DataLoader
import json

# -------------------------------
# Load multilingual model
# -------------------------------
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# -------------------------------
# Load dataset
# -------------------------------
with open('./data/train_dataset.json', 'r', encoding='utf-8') as f:
    train_dataset = json.load(f)

with open('./data/val_dataset.json', 'r', encoding='utf-8') as f:
    val_dataset = json.load(f)

# -------------------------------
# Build InputExample objects
# -------------------------------
def build_examples(dataset):
    corpus = dataset['corpus']
    queries = dataset['queries']
    relevant_docs = dataset['relevant_docs']
    examples = []
    for qid, query in queries.items():
        doc_id = relevant_docs[qid][0]
        text = corpus[doc_id]
        examples.append(InputExample(texts=[query, text]))
    return examples

train_examples = build_examples(train_dataset)
val_examples   = build_examples(val_dataset)

BATCH_SIZE = 8
train_loader = DataLoader(train_examples, batch_size=BATCH_SIZE, shuffle=True)

# -------------------------------
# Loss function
# -------------------------------
loss = losses.MultipleNegativesRankingLoss(model)

# -------------------------------
# Evaluator
# -------------------------------
evaluator = InformationRetrievalEvaluator(
    queries=val_dataset['queries'],
    corpus=val_dataset['corpus'],
    relevant_docs=val_dataset['relevant_docs'],
    show_progress_bar=True
)

# -------------------------------
# Train / Fine-tune
# -------------------------------
EPOCHS = 2
warmup_steps = int(len(train_loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(train_loader, loss)],
    evaluator=evaluator,
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='exp_finetune',
    show_progress_bar=True,
    evaluation_steps=50
)


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


...

Corpus Chunks: 100%|██████████| 1/1 [00:02<00:00,  2.19s/it]


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Corpus Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 1/1 [00:02<00:00,  2.22s/it]


In [ ]:
model.save("paraphrase-multilingual-MiniLM-L12-v2-fine-tuned")

In [ ]:
import shutil

# Folder of the fine-tuned model
model_save_path = '/content/paraphrase-multilingual-MiniLM-L12-v2-fine-tuned'

# Where to save the zip
zip_path = '/content/paraphrase-multilingual-MiniLM-L12-v2-fine-tuned-sentence.zip'

# Zip only the model folder
shutil.make_archive(zip_path.replace('.zip',''), 'zip', model_save_path)

print("Model zipped at:", zip_path)


Model zipped at: /content/paraphrase-multilingual-MiniLM-L12-v2-fine-tuned-sentence.zip


In [ ]:

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
drive_path = "/content/drive/MyDrive/Colab Notebooks/AI_task/RAG"
!cp "{zip_path}" "{drive_path}"
print("Copied zip to Google Drive")

Copied zip to Google Drive


### Evaluate

📊 Explanation of Metrics

Here’s a simple explanation you can paste into your Colab as a text cell:

Exact Match (EM)

Checks if the top-1 retrieved chunk is exactly the gold/correct answer.

Strictest metric.

Example: If the gold answer is in top-1 → EM=1, else 0.

Precision@k

Of the top-𝑘 retrieved chunks, how many are relevant?

Formula:

\text{Precision@k} = \frac{\text{# relevant retrieved in top k}}{k}

Example: Query retrieves 5 chunks, 2 are correct → Precision@5 = 2/5 = 0.4.

Precision focuses on quality of top results.

Recall@k

Of all possible relevant chunks, how many did we retrieve in the top-𝑘?

Formula:

\text{Recall@k} = \frac{\text{# relevant retrieved in top k}}{\text{# total relevant}}

Example: If there are 4 relevant docs, and top-10 retrieved 3 → Recall@10 = 3/4 = 0.75.

Recall focuses on coverage.

Hit@k

Simple: Did we retrieve at least one relevant chunk in top-𝑘?

Binary (1 if yes, 0 if no).

Example: If gold answer is in top-3 → Hit@3=1.

MRR (Mean Reciprocal Rank)

Measures how early the first relevant chunk appears.

Formula:


MRR = mean(
1
rank of first correct doc
)
MRR=mean(
rank of first correct doc
1
	​

)

Example: If correct doc is always rank 1 → MRR=1.

If correct doc is always rank 10 → MRR=0.1.

MAP@k (Mean Average Precision)

Combines precision at multiple ranks up to 𝑘.

Rewards systems that retrieve all relevant documents, and puts more weight if they appear early.

Example: If relevant docs are scattered in top-10, MAP will be lower than if they’re all ranked at top.

In [ ]:
# Save per-model CSVs to Drive and also write a combined CSV
# Paste into Colab. If rank_bm25 missing: !pip install rank_bm25
import os
import json
import re
from typing import List, Dict, Tuple, Any
import numpy as np
import pandas as pd
from tqdm import tqdm

from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi

# ---------------- Config (edit if needed) ----------------
VAL_DATA_PATH = "./data/val_dataset.json"
BASELINE_MODEL_ID = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
FINE_TUNED_MODEL_PATH = "paraphrase-multilingual-MiniLM-L12-v2-fine-tuned"
# Drive output directory (will be created)
OUT_DIR_DRIVE = "/content/drive/MyDrive/Colab Notebooks/AI_task/RAG/retrieval_results"
TOPK = [1, 3, 5, 10]
BATCH_SIZE = 64

# ---------------- Mount Drive (Colab) ----------------
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
except Exception as e:
    # not running in Colab or drive lib missing; continue and write to local if needed
    print("[WARN] google.colab.drive not available or mount failed:", e)

os.makedirs(OUT_DIR_DRIVE, exist_ok=True)
print("Output directory:", OUT_DIR_DRIVE)

# ---------------- Utilities ----------------
def load_dataset(path: str) -> Tuple[Dict[str,str], Dict[str,str], Dict[str, List[str]]]:
    with open(path, "r", encoding="utf-8") as f:
        d = json.load(f)
    return d["queries"], d["corpus"], d["relevant_docs"]

def _normalize_whitespace(s: str) -> str:
    return re.sub(r'\s+', ' ', s).strip()

def simple_normalize(text: str) -> str:
    text = text.replace('\ufeff', ' ')
    text = re.sub(r'[\u200c\u200b\u200d\u200e\u200f]+', ' ', text)
    text = text.replace('ي', 'ی').replace('ك', 'ک')
    text = re.sub(r'[\x00-\x1f]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

_TOKEN_RE = re.compile(r"(?u)\b\w+\b")
def tokenize_for_bm25(text: str) -> List[str]:
    text = simple_normalize(text).lower()
    return _TOKEN_RE.findall(text)

def _tokenize_simple(s: str) -> List[str]:
    return re.findall(r"(?u)\b\w+\b", s)

# ---------------- Metrics ----------------
def compute_metrics_for_k(retrieved_ids: List[List[str]],
                          relevant_map: Dict[str, List[str]],
                          qids: List[str],
                          k: int):
    valid_qids = [q for q in qids if len(relevant_map.get(q, []))>0]
    m = len(valid_qids)
    if m == 0:
        return {"hit@k": None, "precision@k": None, "recall@k": None, "mrr": None, "map@k": None}
    hits = 0
    precision_sum = 0.0
    recall_sum = 0.0
    rr_sum = 0.0
    ap_sum = 0.0
    for i, qid in enumerate(qids):
        relevant = set(relevant_map.get(qid, []))
        if len(relevant) == 0:
            continue
        retrieved = retrieved_ids[i][:k]
        is_hit = len(set(retrieved) & relevant) > 0
        if is_hit:
            hits += 1
        rel_retrieved = len(set(retrieved) & relevant)
        precision_sum += rel_retrieved / float(k)
        recall_sum += rel_retrieved / float(len(relevant))
        rr = 0.0
        ap = 0.0
        first_rel_rank = None
        num_rel_found = 0
        for rank_idx, docid in enumerate(retrieved, start=1):
            if docid in relevant:
                num_rel_found += 1
                if first_rel_rank is None:
                    first_rel_rank = rank_idx
                ap += (num_rel_found / rank_idx)
        if first_rel_rank is not None:
            rr = 1.0 / float(first_rel_rank)
            ap = ap / float(len(relevant))
        else:
            rr = 0.0
            ap = 0.0
        rr_sum += rr
        ap_sum += ap
    return {
        "hit@k": hits / m,
        "precision@k": precision_sum / m,
        "recall@k": recall_sum / m,
        "mrr": rr_sum / m,
        "map@k": ap_sum / m
    }

# ---------------- Embedding retrieval ----------------
def encode_texts(model: SentenceTransformer, texts: List[str], batch_size:int=64, normalize:bool=True) -> np.ndarray:
    embs = model.encode(texts, batch_size=batch_size, show_progress_bar=True, convert_to_numpy=True)
    if normalize:
        norms = np.linalg.norm(embs, axis=1, keepdims=True)
        norms[norms==0] = 1.0
        embs = embs / norms
    return embs

def embedding_retrieve_all(model_path: str, queries: Dict[str,str], corpus: Dict[str,str], batch_size:int=64, max_k:int=10):
    model = SentenceTransformer(model_path)
    qids = list(queries.keys())
    corpus_ids = list(corpus.keys())
    query_texts = [queries[q] for q in qids]
    corpus_texts = [corpus[cid] for cid in corpus_ids]

    print(f"Encoding corpus for model {model_path} ...")
    corpus_embs = encode_texts(model, corpus_texts, batch_size=batch_size, normalize=True)
    print(f"Encoding queries for model {model_path} ...")
    query_embs = encode_texts(model, query_texts, batch_size=batch_size, normalize=True)

    sims = np.dot(query_embs, corpus_embs.T)
    topk_idx = np.argsort(-sims, axis=1)[:, :max_k]
    topk_scores = np.take_along_axis(sims, topk_idx, axis=1)
    retrieved_ids = [[corpus_ids[i] for i in row] for row in topk_idx.tolist()]
    return qids, retrieved_ids, topk_scores, corpus_ids

# ---------------- TF-IDF retrieval ----------------
def build_tfidf_and_retrieve(queries: Dict[str,str], corpus: Dict[str,str], max_k:int=10, tfidf_params: dict=None):
    tfidf_params = tfidf_params or {
        "lowercase": True,
        "analyzer": "word",
        "token_pattern": r"(?u)\b\w+\b",
        "ngram_range": (1,2),
        "max_features": None
    }
    corpus_ids = list(corpus.keys())
    corpus_texts = [ _normalize_whitespace(corpus[cid]) for cid in corpus_ids ]
    vectorizer = TfidfVectorizer(**tfidf_params)
    tfidf_matrix = vectorizer.fit_transform(corpus_texts)

    qids = list(queries.keys())
    query_texts = [queries[qid] for qid in qids]
    q_vecs = vectorizer.transform(query_texts)
    sims = cosine_similarity(q_vecs, tfidf_matrix)
    topk_idx = np.argsort(-sims, axis=1)[:, :max_k]
    topk_scores = np.take_along_axis(sims, topk_idx, axis=1)
    retrieved_ids = [[corpus_ids[i] for i in row] for row in topk_idx.tolist()]
    return qids, retrieved_ids, topk_scores, corpus_ids, vectorizer, tfidf_matrix

# ---------------- BM25 retrieval ----------------
def build_bm25_and_retrieve(queries: Dict[str,str], corpus: Dict[str,str], max_k:int=10):
    corpus_ids = list(corpus.keys())
    corpus_texts = [corpus[cid] for cid in corpus_ids]
    tokenized_corpus = [tokenize_for_bm25(t) for t in corpus_texts]
    bm25 = BM25Okapi(tokenized_corpus, k1=1.5, b=0.75)

    qids = list(queries.keys())
    retrieved_ids = []
    retrieved_scores = []
    for qid in tqdm(qids, desc="BM25 retrieving"):
        qtext = queries[qid]
        q_tokens = tokenize_for_bm25(qtext)
        scores = bm25.get_scores(q_tokens)
        top_idx = np.argsort(-scores)[:max_k]
        top_scores = scores[top_idx]
        top_ids = [corpus_ids[i] for i in top_idx.tolist()]
        retrieved_ids.append(top_ids)
        retrieved_scores.append(top_scores)
    retrieved_scores = np.array(retrieved_scores)
    return qids, retrieved_ids, retrieved_scores, corpus_ids, bm25, tokenized_corpus

# ---------------- Helper: safe filename ----------------
def safe_label(s: str) -> str:
    safe = re.sub(r'[^a-zA-Z0-9_\-\.]', '_', s)
    return safe

# ---------------- Unified evaluation driver (writes per-model CSV) ----------------
def evaluate_retriever_and_save(retriever_type: str, val_data_path: str,
                       model_path: str = None,
                       model_name: str = None,
                       topk_list: List[int] = [1,3,5,10],
                       batch_size: int = 64,
                       tfidf_params: dict = None,
                       out_dir_drive: str = OUT_DIR_DRIVE):
    queries, corpus, relevant = load_dataset(val_data_path)
    max_k = max(topk_list)

    if retriever_type == "embedding":
        assert model_path is not None, "provide model_path for embedding retriever"
        qids, retrieved_ids, retrieved_scores, corpus_ids = embedding_retrieve_all(model_path, queries, corpus, batch_size=batch_size, max_k=max_k)
        model_label = model_name or model_path
    elif retriever_type == "tfidf":
        qids, retrieved_ids, retrieved_scores, corpus_ids, _, _ = build_tfidf_and_retrieve(queries, corpus, max_k, tfidf_params=tfidf_params)
        model_label = "tfidf"
    elif retriever_type == "bm25":
        qids, retrieved_ids, retrieved_scores, corpus_ids, bm25_obj, tokenized_corpus = build_bm25_and_retrieve(queries, corpus, max_k)
        model_label = "bm25"
    else:
        raise ValueError("retriever_type must be 'embedding' or 'tfidf' or 'bm25'")

    # compute metrics for each k and build DataFrame rows
    rows = []
    results = {}
    for k in topk_list:
        m = compute_metrics_for_k(retrieved_ids, relevant, qids, k)
        if k == 1:
            m["EM"] = m["hit@k"]
        results[k] = m
        rows.append({
            "retriever": retriever_type,
            "model": model_label,
            "top_k": k,
            "EM": m.get("EM", None),
            "hit@k": m["hit@k"],
            "precision@k": m["precision@k"],
            "recall@k": m["recall@k"],
            "mrr": m["mrr"],
            "map@k": m["map@k"]
        })
    df = pd.DataFrame(rows)

    # write per-model CSV (overwrite)
    safe_name = safe_label(model_label)
    per_model_csv = os.path.join(out_dir_drive, f"{safe_name}_retrieval_eval.csv")
    df.to_csv(per_model_csv, index=False, encoding="utf-8")
    print("Saved per-model CSV:", per_model_csv)

    return {
        "retriever": retriever_type,
        "model": model_label,
        "qids": qids,
        "corpus_ids": corpus_ids,
        "retrieved_ids": retrieved_ids,
        "retrieved_scores": retrieved_scores,
        "results": results,
        "df_summary": df,
        "per_model_csv": per_model_csv
    }

# ---------------- Run evaluations and save combined CSV ----------------
if __name__ == "__main__":
    assert os.path.exists(VAL_DATA_PATH), f"val dataset missing at {VAL_DATA_PATH}"

    outputs = []

    # Baseline embedding
    print("Evaluating baseline embedding model:", BASELINE_MODEL_ID)
    out_base = evaluate_retriever_and_save(
        "embedding",
        VAL_DATA_PATH,
        model_path=BASELINE_MODEL_ID,
        model_name="baseline_paraphrase_miniLM",
        topk_list=TOPK,
        batch_size=BATCH_SIZE
    )
    outputs.append(out_base["df_summary"])

    # Fine-tuned embedding
    print("Evaluating fine-tuned embedding model:", FINE_TUNED_MODEL_PATH)
    out_ft = evaluate_retriever_and_save(
        "embedding",
        VAL_DATA_PATH,
        model_path=FINE_TUNED_MODEL_PATH,
        model_name="fine_tuned_paraphrase_miniLM",
        topk_list=TOPK,
        batch_size=BATCH_SIZE
    )
    outputs.append(out_ft["df_summary"])

    # TF-IDF
    print("Evaluating TF-IDF")
    out_tfidf = evaluate_retriever_and_save(
        "tfidf",
        VAL_DATA_PATH,
        model_path=None,
        model_name="tfidf",
        topk_list=TOPK,
        batch_size=BATCH_SIZE
    )
    outputs.append(out_tfidf["df_summary"])

    # BM25
    print("Evaluating BM25")
    out_bm25 = evaluate_retriever_and_save(
        "bm25",
        VAL_DATA_PATH,
        model_path=None,
        model_name="bm25",
        topk_list=TOPK,
        batch_size=BATCH_SIZE
    )
    outputs.append(out_bm25["df_summary"])

    # combined CSV
    combined_df = pd.concat(outputs, ignore_index=True)
    combined_csv = os.path.join(OUT_DIR_DRIVE, "combined_retrieval_results.csv")
    combined_df.to_csv(combined_csv, index=False, encoding="utf-8")
    print("Saved combined CSV:", combined_csv)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Output directory: /content/drive/MyDrive/Colab Notebooks/AI_task/RAG/retrieval_results
Evaluating baseline embedding model: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Encoding corpus for model sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 ...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Encoding queries for model sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 ...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Saved per-model CSV: /content/drive/MyDrive/Colab Notebooks/AI_task/RAG/retrieval_results/baseline_paraphrase_miniLM_retrieval_eval.csv
Evaluating fine-tuned embedding model: paraphrase-multilingual-MiniLM-L12-v2-fine-tuned
Encoding corpus for model paraphrase-multilingual-MiniLM-L12-v2-fine-tuned ...


...

Encoding queries for model paraphrase-multilingual-MiniLM-L12-v2-fine-tuned ...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Saved per-model CSV: /content/drive/MyDrive/Colab Notebooks/AI_task/RAG/retrieval_results/fine_tuned_paraphrase_miniLM_retrieval_eval.csv
Evaluating TF-IDF
Saved per-model CSV: /content/drive/MyDrive/Colab Notebooks/AI_task/RAG/retrieval_results/tfidf_retrieval_eval.csv
Evaluating BM25


BM25 retrieving: 100%|██████████| 79/79 [00:00<00:00, 3654.02it/s]


Saved per-model CSV: /content/drive/MyDrive/Colab Notebooks/AI_task/RAG/retrieval_results/bm25_retrieval_eval.csv
Saved combined CSV: /content/drive/MyDrive/Colab Notebooks/AI_task/RAG/retrieval_results/combined_retrieval_results.csv


### Rsults

In [ ]:
import pandas as pd

In [ ]:
word_chunking_results = pd.read_csv('/content/combined_retrieval_results.csv') #word-based chunking
sentence_chunking_results = pd.read_csv('/content/combined_retrieval_results(sentence).csv') #sentence-based chunking


In [ ]:
word_chunking_results

,retriever,model,top_k,EM,hit@k,precision@k,recall@k,mrr,map@k
0,embedding,baseline_paraphrase_miniLM,1,0.515789,0.515789,0.515789,0.515789,0.515789,0.515789
1,embedding,baseline_paraphrase_miniLM,3,NaN,0.715789,0.238596,0.715789,0.612281,0.612281
2,embedding,baseline_paraphrase_miniLM,5,NaN,0.789474,0.157895,0.789474,0.629123,0.629123
3,embedding,baseline_paraphrase_miniLM,10,NaN,0.873684,0.087368,0.873684,0.640610,0.640610
4,embedding,fine_tuned_paraphrase_miniLM,1,0.494737,0.494737,0.494737,0.494737,0.494737,0.494737
5,embedding,fine_tuned_paraphrase_miniLM,3,NaN,0.694737,0.231579,0.694737,0.585965,0.585965
6,embedding,fine_tuned_paraphrase_miniLM,5,NaN,0.747368,0.149474,0.747368,0.598596,0.598596
7,embedding,fine_tuned_paraphrase_miniLM,10,NaN,0.884211,0.088421,0.884211,0.616395,0.616395
8,tfidf,tfidf,1,0.684211,0.684211,0.684211,0.684211,0.684211,0.684211
9,tfidf,tfidf,3,NaN,0.852632,0.284211,0.852632,0.764912,0.764912


In [ ]:
sentence_chunking_results

,retriever,model,top_k,EM,hit@k,precision@k,recall@k,mrr,map@k
0,embedding,baseline_paraphrase_miniLM,1,0.506329,0.506329,0.506329,0.506329,0.506329,0.506329
1,embedding,baseline_paraphrase_miniLM,3,NaN,0.696203,0.232068,0.696203,0.590717,0.590717
2,embedding,baseline_paraphrase_miniLM,5,NaN,0.772152,0.154430,0.772152,0.607173,0.607173
3,embedding,baseline_paraphrase_miniLM,10,NaN,0.860759,0.086076,0.860759,0.617877,0.617877
4,embedding,fine_tuned_paraphrase_miniLM,1,0.468354,0.468354,0.468354,0.468354,0.468354,0.468354
5,embedding,fine_tuned_paraphrase_miniLM,3,NaN,0.670886,0.223629,0.670886,0.565401,0.565401
6,embedding,fine_tuned_paraphrase_miniLM,5,NaN,0.797468,0.159494,0.797468,0.593882,0.593882
7,embedding,fine_tuned_paraphrase_miniLM,10,NaN,0.886076,0.088608,0.886076,0.605129,0.605129
8,tfidf,tfidf,1,0.797468,0.797468,0.797468,0.797468,0.797468,0.797468
9,tfidf,tfidf,3,NaN,0.936709,0.312236,0.936709,0.864979,0.864979
